In [33]:
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import torch
from tqdm import tqdm

import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import copy
import datetime as dt

In [34]:
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [35]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [36]:
train_df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [37]:
test_df.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1


In [38]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1000 non-null   int64  
 1   battery_power  1000 non-null   int64  
 2   blue           1000 non-null   int64  
 3   clock_speed    1000 non-null   float64
 4   dual_sim       1000 non-null   int64  
 5   fc             1000 non-null   int64  
 6   four_g         1000 non-null   int64  
 7   int_memory     1000 non-null   int64  
 8   m_dep          1000 non-null   float64
 9   mobile_wt      1000 non-null   int64  
 10  n_cores        1000 non-null   int64  
 11  pc             1000 non-null   int64  
 12  px_height      1000 non-null   int64  
 13  px_width       1000 non-null   int64  
 14  ram            1000 non-null   int64  
 15  sc_h           1000 non-null   int64  
 16  sc_w           1000 non-null   int64  
 17  talk_time      1000 non-null   int64  
 18  three_g  

In [40]:
X = train_df.drop(columns = ['price_range'])
y = train_df['price_range']

In [41]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(test_df.drop(columns = ['id']))

In [44]:
pd.DataFrame(X_val_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.915888,-0.987224,1.181201,-1.048308,-0.293030,0.945755,-0.396739,0.324055,1.680880,-1.118271,-0.814054,-0.974147,0.840585,-1.327357,-0.991123,0.079727,0.009370,0.555348,0.991465,-0.998572
1,-0.131719,-0.987224,-1.243552,-1.048308,0.632502,0.945755,-1.328218,-0.023324,-0.074820,1.496111,1.013414,-0.828623,-0.605127,0.406926,1.613980,2.600856,1.466977,0.555348,-1.008608,-0.998572
2,1.651923,-0.987224,1.666151,-1.048308,1.095268,-1.057357,-0.999461,-0.370703,1.567609,1.060381,1.345682,-0.787694,-0.684153,-0.745259,-0.991123,-1.066241,-0.537232,0.555348,0.991465,-0.998572
3,-0.567469,1.012941,0.575013,-1.048308,-0.061647,-1.057357,-0.835082,-1.065461,0.718077,-0.682540,1.511815,-0.871825,0.343186,1.634877,1.377152,0.308920,1.466977,0.555348,0.991465,-0.998572
4,-1.411876,1.012941,-1.243552,0.953918,0.632502,-1.057357,1.411425,-0.023324,-0.301361,0.188920,-0.315654,0.867638,1.654089,-0.200119,0.429842,-0.149467,-1.083835,0.555348,-1.008608,-0.998572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,-0.054954,-0.987224,1.787389,-1.048308,-0.061647,-1.057357,0.808703,-0.370703,1.652563,-0.682540,-0.481787,2.757173,1.675008,0.134357,0.193014,-0.149467,-1.266036,-1.800673,0.991465,1.001430
596,1.618056,-0.987224,-0.394888,0.953918,3.177714,0.945755,0.205982,1.018813,0.293311,-1.118271,1.511815,1.261007,1.117177,-0.930976,0.903497,-0.837048,1.284776,0.555348,-1.008608,1.001430
597,-1.547343,1.012941,-1.243552,0.953918,-0.524413,-1.057357,-0.835082,-1.065461,0.633124,0.188920,0.348880,0.719841,-0.549344,-1.644278,1.613980,1.913275,-1.448236,-1.800673,0.991465,1.001430
598,-0.434260,1.012941,-0.031175,-1.048308,-0.524413,0.945755,-0.177568,1.713570,1.171161,-1.554001,-0.980188,-1.387980,0.984692,0.021633,-0.280641,0.996501,-1.630437,0.555348,-1.008608,-0.998572


In [45]:
pd.DataFrame(X_test_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.445549,1.012941,0.332537,0.953918,2.252182,-1.057357,-1.492596,-1.412840,1.482656,-0.682540,1.013414,-0.940039,0.385023,1.250507,-0.043813,0.308920,-1.630437,-1.800673,0.991465,-0.998572
1,-0.901619,1.012941,-1.243552,0.953918,-0.061647,0.945755,1.575803,1.018813,1.426021,0.188920,0.348880,0.242341,-0.904962,1.637649,-1.464779,-1.295435,-0.719433,0.555348,-1.008608,-0.998572
2,1.279390,1.012941,1.544914,-1.048308,-0.755796,-1.057357,-0.287154,1.366192,1.284432,-0.682540,-0.980188,1.433816,0.278105,0.252624,1.140325,0.996501,-0.172831,-1.800673,0.991465,1.001430
3,0.690111,-0.987224,-1.243552,0.953918,3.177714,0.945755,-0.396739,-0.023324,-1.264165,1.496111,1.677949,-0.783147,1.175284,1.635801,-0.517468,-1.295435,-0.719433,0.555348,0.991465,-0.998572
4,0.437240,-0.987224,-0.152413,-1.048308,1.558033,0.945755,0.918289,-0.023324,-0.924352,0.624650,1.345682,0.249162,-1.014204,-0.323007,0.666670,0.538114,-0.719433,0.555348,-1.008608,1.001430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.037808,1.012941,0.453775,-1.048308,-0.987179,0.945755,1.192253,-0.023324,0.831348,1.060381,1.179548,0.010412,-0.774801,-0.001466,0.429842,0.538114,0.738174,0.555348,0.991465,-0.998572
996,-1.425423,-0.987224,0.332537,0.953918,-0.987179,-1.057357,-1.054254,1.366192,1.284432,-0.246810,-1.312455,1.165507,0.896368,-0.175172,-0.991123,-1.066241,1.466977,-1.800673,0.991465,1.001430
997,-0.124945,-0.987224,-0.152413,-1.048308,-0.755796,0.945755,-1.328218,-0.023324,-1.717248,-1.554001,0.348880,-0.369314,-0.979339,-0.831188,-1.701606,-1.295435,0.555973,0.555348,-1.008608,-0.998572
998,0.660760,1.012941,-1.243552,0.953918,-0.987179,-1.057357,0.973082,-0.370703,0.859666,-1.118271,0.348880,-1.367515,-0.963069,0.357032,0.666670,1.225694,-0.901634,-1.800673,0.991465,-0.998572


In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rfc = RandomForestClassifier()

rfc.fit(X_train_scaled, y_train)

RandomForestClassifier()

In [47]:
pred = rfc.predict(X_val_scaled)

In [48]:
pred

array([0, 1, 1, 3, 1, 2, 2, 0, 3, 1, 0, 0, 2, 3, 2, 2, 3, 3, 1, 0, 0, 1,
       1, 2, 0, 1, 3, 2, 2, 0, 0, 0, 3, 0, 1, 1, 2, 0, 3, 0, 2, 3, 2, 0,
       3, 2, 2, 1, 3, 1, 3, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 3, 3, 1, 0, 0,
       3, 3, 1, 2, 2, 3, 0, 1, 2, 0, 0, 3, 2, 2, 3, 2, 1, 0, 1, 3, 2, 3,
       3, 0, 3, 3, 2, 1, 3, 2, 2, 3, 1, 1, 0, 0, 1, 0, 0, 3, 2, 0, 1, 1,
       0, 0, 3, 1, 2, 2, 2, 3, 0, 2, 1, 3, 2, 1, 3, 3, 0, 2, 0, 2, 3, 0,
       2, 2, 0, 3, 1, 0, 0, 2, 3, 0, 2, 3, 0, 0, 0, 1, 1, 2, 3, 1, 1, 0,
       2, 2, 0, 1, 0, 2, 2, 3, 2, 2, 1, 0, 0, 2, 2, 3, 3, 1, 0, 0, 3, 1,
       2, 2, 1, 0, 0, 0, 0, 0, 3, 2, 0, 3, 0, 0, 0, 0, 1, 3, 3, 1, 0, 1,
       1, 1, 1, 2, 1, 2, 3, 3, 1, 3, 0, 0, 0, 2, 1, 1, 3, 1, 0, 2, 1, 1,
       3, 1, 3, 0, 0, 2, 1, 2, 0, 0, 2, 0, 1, 3, 2, 0, 1, 3, 3, 0, 1, 3,
       3, 3, 0, 3, 1, 2, 3, 3, 2, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 1, 2,
       2, 2, 2, 0, 2, 3, 2, 2, 2, 1, 0, 2, 0, 2, 3, 1, 3, 1, 0, 3, 1, 2,
       0, 0, 3, 0, 1, 2, 3, 3, 3, 1, 1, 0, 1, 3, 3,

In [49]:
accuracy_score(y_val, pred)

0.865